mounting drive for dataset

In [17]:
from google.colab import drive
drive.mount ("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
import os
os.listdir("/content/drive")

['MyDrive', 'Shareddrives', '.shortcut-targets-by-id', '.Trash-0']

In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cpu


In [23]:
dataset_path ="/content/drive/My Drive/mrl_eye_dataset/data"

train_data=f"{dataset_path}/train"
val_data=f"{dataset_path}/validate"

In [24]:
from torchvision import transforms

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

In [26]:
from torchvision.datasets import ImageFolder

train_dataset = ImageFolder(root=train_data, transform=transform)
val_dataset   = ImageFolder(root=val_data,   transform=transform)

In [30]:
from torch.utils.data import DataLoader

BATCH_SIZE = 64

train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=2
)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=2
)


In [31]:
images, labels = next(iter(train_loader))

print("Images shape:", images.shape)
print("Labels shape:", labels.shape)
print("Unique labels:", labels.unique())

Images shape: torch.Size([64, 3, 64, 64])
Labels shape: torch.Size([64])
Unique labels: tensor([0, 1])


In [32]:
import torch
import torch.nn as nn
from torchvision import models

#  loading model EfficientNet-B0
model = models.efficientnet_b0(pretrained=True)

# Replace the classifier head (1000 classes → 2 classes)
in_features = model.classifier[1].in_features
model.classifier[1] = nn.Linear(in_features, 2)

# Move model to device
model = model.to(device)

# Sanity check
print(model.classifier)


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth


100%|██████████| 20.5M/20.5M [00:00<00:00, 41.3MB/s]


Sequential(
  (0): Dropout(p=0.2, inplace=True)
  (1): Linear(in_features=1280, out_features=2, bias=True)
)
